Importing libraries

In [1]:
from bs4 import BeautifulSoup
import requests

Getting the table from the website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Getting the headings of the table

In [3]:
table = soup.find('table',class_ = 'wikitable sortable')
head = []
for heading in table.find_all('th'):
    head.append(heading.text)
head[2] = head[2].replace('\n','')
head[1] = head[1].replace('\n','')
print(head)

['Postcode', 'Borough', 'Neighbourhood']


Crearing a dataframe

In [4]:
import pandas as pd
df = pd.DataFrame(columns=head)
print(df)

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


Inserting values

In [5]:
for row in table.find_all('tr'):
    temp=[]
    for data in row.find_all('td'):
        temp.append(data.text)
    if len(temp) > 0:
        df = df.append({'Postcode':temp[0],'Borough':temp[1],'Neighbourhood':temp[2]},ignore_index=True)

In [6]:
print(df.head())

  Postcode           Borough       Neighbourhood
0      M1A      Not assigned      Not assigned\n
1      M2A      Not assigned      Not assigned\n
2      M3A        North York         Parkwoods\n
3      M4A        North York  Victoria Village\n
4      M5A  Downtown Toronto      Harbourfront\n


Cleaning the data

In [7]:
n = df['Neighbourhood'].values
new=[]
for i in n:
    #print(i)
    i = i.replace('\n','')
    new.append(i)
#print(new)
df['Neighbourhood'] = new
#print(df.head())

n = df['Borough'].values
new=[]
for i in n:
    #print(i)
    i = i.replace('\n','')
    new.append(i)
#print(new)
df['Borough'] = new
print(df.head())

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


Removing rows where Borough is not assigned

In [8]:
df = df[df['Borough']!='Not assigned']
print(df.head())

  Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights


Grouping rows by Postcode

In [9]:
foo = lambda a: ','.join(a)
df = df.groupby('Postcode',as_index=False).agg({'Borough':'first','Neighbourhood':foo}).reindex(columns=df.columns)
print(df.head())

  Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae


In [10]:
print(df.shape)

(103, 3)


In [11]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df['latitude'] = df2['Latitude'].values
df['longitude'] = df2['Longitude'].values
print(df.head())

  Postcode      Borough                         Neighbourhood   latitude  \
0      M1B  Scarborough                         Rouge,Malvern  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  


Clustring and analyzing only those rows where Borough is Downtown Toronto

In [12]:
df = df[df['Borough']=='Downtown Toronto'].reset_index(drop=True)
print(df)

   Postcode           Borough  \
0       M4W  Downtown Toronto   
1       M4X  Downtown Toronto   
2       M4Y  Downtown Toronto   
3       M5A  Downtown Toronto   
4       M5B  Downtown Toronto   
5       M5C  Downtown Toronto   
6       M5E  Downtown Toronto   
7       M5G  Downtown Toronto   
8       M5H  Downtown Toronto   
9       M5J  Downtown Toronto   
10      M5K  Downtown Toronto   
11      M5L  Downtown Toronto   
12      M5S  Downtown Toronto   
13      M5T  Downtown Toronto   
14      M5V  Downtown Toronto   
15      M5W  Downtown Toronto   
16      M5X  Downtown Toronto   
17      M6G  Downtown Toronto   

                                        Neighbourhood   latitude  longitude  
0                                            Rosedale  43.679563 -79.377529  
1                          Cabbagetown,St. James Town  43.667967 -79.367675  
2                                Church and Wellesley  43.665860 -79.383160  
3                            Harbourfront,Regent Park  43.65

In [13]:
from geopy.geocoders import Nominatim
address = 'Downtown Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [14]:
CLIENT_ID = 'DZKMKDK3NUILPHP03BQ41RD34VVEKLT5GENKLABJLOOJ204I' # your Foursquare ID
CLIENT_SECRET = 'USUJ2ATZ5I43HBIHFNCVXYAIG4ZJBSFB0SAYEGZPOLHQZUD0' # your Foursquare Secret
VERSION = '20180605' 
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
downtown_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'])
print(downtown_venues)

                              Neighbourhood  Neighbourhood Latitude  \
0                                  Rosedale               43.679563   
1                                  Rosedale               43.679563   
2                                  Rosedale               43.679563   
3                                  Rosedale               43.679563   
4                Cabbagetown,St. James Town               43.667967   
5                Cabbagetown,St. James Town               43.667967   
6                Cabbagetown,St. James Town               43.667967   
7                Cabbagetown,St. James Town               43.667967   
8                Cabbagetown,St. James Town               43.667967   
9                Cabbagetown,St. James Town               43.667967   
10               Cabbagetown,St. James Town               43.667967   
11               Cabbagetown,St. James Town               43.667967   
12               Cabbagetown,St. James Town               43.667967   
13    

In [16]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [17]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"Cabbagetown,St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022727,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00,0.00,0.00,0.011364,0.000000,0.000000,0.011364,0.000000,0.011364
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.01,0.01,0.00,0.00,0.060000,0.000000,0.030000,0.010000,0.000000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00,0.00,0.00,0.000000,0.011364,0.011364,0.000000,0.011364,0.011364
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,...,0.00,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Gym,Bar,Hotel,Burger Joint,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Café,Steakhouse,Seafood Restaurant,Farmers Market,Bakery,Cheese Shop
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Plane,Boutique,Sculpture Garden,Airport Gate,Airport Food Court,Airport
3,"Cabbagetown,St. James Town",Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Pub,Market,Bakery,Café,Pharmacy,Indian Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Salad Place,Spa,Indian Restaurant,Japanese Restaurant,Sushi Restaurant,Middle Eastern Restaurant


In [21]:
from sklearn.cluster import KMeans
import folium
# set number of clusters
kclusters = 3

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0])

In [22]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtownn_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtownn_merged = downtownn_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtownn_merged 

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Pub,Market,Bakery,Café,Pharmacy,Indian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Bubble Tea Shop,Men's Store,Mediterranean Restaurant,Café
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Event Space
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Bakery,Italian Restaurant,Pizza Place,Bookstore
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Cosmetics Shop,Italian Restaurant,Bakery,Breakfast Spot,Gastropub,Beer Bar
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Café,Steakhouse,Seafood Restaurant,Farmers Market,Bakery,Cheese Shop
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Salad Place,Spa,Indian Restaurant,Japanese Restaurant,Sushi Restaurant,Middle Eastern Restaurant
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Gym,Bar,Hotel,Burger Joint,Bakery
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Restaurant,Bakery,Pizza Place,Scenic Lookout,Fried Chicken Joint
